# 1. Cargar Librerías

In [ ]:
import warnings
import os
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import datasets as ds
import math
import utils as ut
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings("ignore", category=FutureWarning)

# 2. Cargar los Dataset

In [ ]:
movies_df = pd.read_csv("../data/raw/rotten_tomatoes_movies.csv")

movies_df.head()

In [ ]:
critics_df = pd.read_csv("../data/raw/rotten_tomatoes_critic_reviews.csv")

critics_df.head()

In [ ]:
ut.analisis(movies_df)

In [ ]:
ut.analisis(critics_df)

El dataset de películas, ***movies_df***, contiene 22 variables: 8 variable numéricas y 14 variables categóricas.

- rotten_tomatoes_link: Identificador o enlace único de la película en Rotten Tomatoes.
- movie_title: Título de la película.
- movie_info: Información general (sinopsis, detalles adicionales).
- critics_consensus: Resumen del consenso de los críticos.
- content_rating: Clasificación de la película (PG, R, etc.).
- genres: Género(s) de la película.
- directors: Director(es) de la película.
- authors: Críticos o autores asociados.
- actors: Actores principales.
- original_release_date: Fecha de estreno original.
- streaming_release_date: Fecha de estreno en plataformas digitales.
- runtime: Duración de la película (minutos).
- production_company: Productora responsable.
- tomatometer_status: Estado en el Tomatometer (Certified Fresh, Fresh, Rotten).
- tomatometer_rating: Puntuación promedio de críticos.
- tomatometer_count: Número de críticas registradas.
- audience_status: Estado de la audiencia (Upright/Spilled).
- audience_rating: Puntuación promedio de la audiencia.
- audience_count: Número de valoraciones de la audiencia.
- tomatometer_top_critics_count: Número de críticas de críticos top.
- tomatometer_fresh_critics_count: Cantidad de críticas positivas.
- tomatometer_rotten_critics_count: Cantidad de críticas negativas.


El dataset de reseñas, ***critics_df***, contiene 8 variables categóricas.

- rotten_tomatoes_link: Sirve como identificador de la película
- critic_name: Nombre del critico que comento la película
- top_critic: Valor booleano que aclara si el crítico es un crítico destacado o no
- publisher_name: nombre de la editorial para la que trabaja el crítico
- review_type: Determina si la reseña es positiva (fresh) o negativa (rotten)
- review_score: Puntaje proporcionado por el crítico
- review_date: Fecha de la reseña
- review_content: Contenido de la reseña

# 2. Exploración y limpieza

### 2.1 Comprensión de Datos

In [ ]:
print(f'El dataframe movies_df contiene {movies_df.shape[0]} filas y {movies_df.shape[1]} columnas.')

In [ ]:
print(f'El dataframe critics_df contiene {critics_df.shape[0]} filas y {critics_df.shape[1]} columnas.')

### 2.2 Identificando nulos y duplicados

In [ ]:
movies_df.info()

In [ ]:
print(movies_df.isnull().sum())

print("Duplicados:", movies_df.duplicated().sum())

In [ ]:
movies_df = movies_df.drop_duplicates()

In [ ]:
critics_df.info()

In [ ]:
print(critics_df.isnull().sum())

print("Duplicados:", critics_df.duplicated().sum())

In [ ]:
critics_df = critics_df.drop_duplicates()
critics_df = critics_df.dropna(subset=['review_content'])

- Los nulos en el nombre del crítico se contrastan con el de publisher name asi que no es grave y se puede mantener
- El de review score se compensa con el de review type ya que determina si es util o no
- El de texto si es necesario arreglarlo para cuando se haga el NLP

### 2.3 Eliminar Información Irrelevante

In [ ]:
critics = critics_df.copy().drop(columns=['review_date', 'review_score', 'critic_name'])

critics.head()

- La fecha no es relevante para el estudio.
- El puntaje tiene sus nulos, además de que ya hay un boolean que dice si la reseña será positiva o negativa.
- El nombre del crítico no hace falta si ya está el de la editorial que lo publica.

### 2.4 Crear Listas
Para poder hacer un mejor análisis de los datos, por cada columna que tenga varios valores, crearemos una lista de esos valores.

In [ ]:
movies = movies_df.copy()

#### Géneros

In [ ]:
movies["genres"] = movies["genres"].str.split(",")

movies["genres"] = movies["genres"].apply(
    lambda lst: [g.strip() for g in lst] if isinstance(lst, list) else []
)

#### Directores

In [ ]:
movies["directors"] = movies["directors"].str.split(",")

movies["directors"] = movies["directors"].apply(
    lambda lst: [g.strip() for g in lst] if isinstance(lst, list) else []
)

#### Autores

In [ ]:
movies["authors"] = movies["authors"].str.split(",")

movies["authors"] = movies["authors"].apply(
    lambda lst: [g.strip() for g in lst] if isinstance(lst, list) else []
)

#### Actores

In [ ]:
movies["actors"] = movies["actors"].str.split(",")

movies["actors"] = movies["actors"].apply(
    lambda lst: [g.strip() for g in lst] if isinstance(lst, list) else []
)

### 2.5 Nuevos Datasets
Para las listas creadas previamente, crearemos nuevos datasets en el que cada fila corresponda a un elemento de la lista. Con estos datasets haremos un análisis univariante de esas variables.

In [ ]:
movie_genre = movies.explode("genres").reset_index(drop=True)

movie_genre = movie_genre.rename(columns={"genres": "genre"})

movie_genre = movie_genre[["rotten_tomatoes_link", "genre"]].drop_duplicates()

In [ ]:
movie_genre.head()

#### Directores

In [ ]:
movie_director = movies.explode("directors").reset_index(drop=True)

movie_director = movie_director.rename(columns={"directors": "director"})

movie_director = movie_director[["rotten_tomatoes_link", "director"]].drop_duplicates()

In [ ]:
movie_director.head()

#### Autores

In [ ]:
movie_author = movies.explode("authors").reset_index(drop=True)

movie_author = movie_author.rename(columns={"authors": "author"})

movie_author = movie_author[["rotten_tomatoes_link", "author"]].drop_duplicates()

In [ ]:
movie_author.head()

#### Actores

In [ ]:
movie_actor = movies.explode("actors").reset_index(drop=True)

movie_actor = movie_actor.rename(columns={"actors": "actor"})

movie_actor = movie_actor[["rotten_tomatoes_link", "actor"]].drop_duplicates()

In [ ]:
movie_actor.head()

### 2.6 Transformamos las fechas en un formato que nos pueda servir para el análisis

In [ ]:
movies["original_release_date"] = pd.to_datetime(movies["original_release_date"], errors="coerce")

#movies["release_timestamp"] = movies["original_release_date"].astype("int64") // 10**9

movies["release_year"] = movies['original_release_date'].dt.year

movies["release_year"] = movies["release_year"].fillna(0).astype(int)

In [ ]:
movies["streaming_release_date"] = pd.to_datetime(movies["streaming_release_date"], errors="coerce")

#movies["streaming_timestamp"] = movies["streaming_release_date"].astype("int64") // 10**9

movies["streaming_release_year"] = movies['streaming_release_date'].dt.year

movies["streaming_release_year"] = movies["streaming_release_year"].fillna(0).astype(int)

In [ ]:
movies.drop(columns=["original_release_date", "streaming_release_date"], inplace=True)

## 3. Análisis Univariante

### 3.1 Variables Numéricas

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# runtime
sns.histplot(ax = axis[0], data = movies, x = "runtime")
sns.boxplot(ax = axis[1], data = movies, x = "runtime")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# tomatometer_rating
sns.histplot(ax = axis[0], data = movies, x = "tomatometer_rating")
sns.boxplot(ax = axis[1], data = movies, x = "tomatometer_rating")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# tomatometer_count
sns.histplot(ax = axis[0], data = movies, x = "tomatometer_count")
sns.boxplot(ax = axis[1], data = movies, x = "tomatometer_count")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# audience_rating
sns.histplot(ax = axis[0], data = movies, x = "audience_rating")
sns.boxplot(ax = axis[1], data = movies, x = "audience_rating")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# audience_count
sns.histplot(ax = axis[0], data = movies, x = "audience_count")
sns.boxplot(ax = axis[1], data = movies, x = "audience_count")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# tomatometer_top_critics_count
sns.histplot(ax = axis[0], data = movies, x = "tomatometer_top_critics_count")
sns.boxplot(ax = axis[1], data = movies, x = "tomatometer_top_critics_count")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# tomatometer_fresh_critics_count
sns.histplot(ax = axis[0], data = movies, x = "tomatometer_fresh_critics_count")
sns.boxplot(ax = axis[1], data = movies, x = "tomatometer_fresh_critics_count")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# tomatometer_rotten_critics_count
sns.histplot(ax = axis[0], data = movies, x = "tomatometer_rotten_critics_count")
sns.boxplot(ax = axis[1], data = movies, x = "tomatometer_rotten_critics_count")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# year
sns.histplot(ax = axis[0], data = movies[movies["release_year"] > 0], x = "release_year")
sns.boxplot(ax = axis[1], data = movies[movies["release_year"] > 0], x = "release_year")

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 4))

# year
sns.histplot(ax = axis[0], data = movies[movies["streaming_release_year"] > 0], x = "streaming_release_year")
sns.boxplot(ax = axis[1], data = movies[movies["streaming_release_year"] > 0], x = "streaming_release_year")

plt.tight_layout()
plt.show()

### 3.2 Variables Categóricas

In [ ]:
#Genres
plt.figure(figsize=(12, 6))
ax = sns.countplot(
    data=movie_genre,
    x="genre",
    order=movie_genre["genre"].value_counts().index,
    palette="Set2"
)

plt.title("Distribución por Género")
ax.tick_params(axis='x', rotation=90, labelsize=8)
plt.tight_layout()
plt.show()

In [ ]:
#Directors
plt.figure(figsize=(12, 6))
ax = sns.countplot(
    data=movie_director,
    x="director",
    order=movie_director["director"].value_counts().index,
    palette="Set2"
)

plt.title("Distribución por Directores")
ax.tick_params(axis='x', rotation=90, labelsize=8)
plt.tight_layout()
plt.show()

Al tener tantos directores, nos enfocaremos en aquellos que hayan dirigido más de 10 películas.

In [ ]:
# Contar películas por director
director_counts = movie_director['director'].value_counts()

# Filtrar solo directores con más de 10 películas
top_directors_over_10 = director_counts[director_counts > 10].index

# Filtrar el dataset
movie_director_over_10 = movie_director[movie_director['director'].isin(top_directors_over_10)]

# Gráfico
plt.figure(figsize=(12, 6))
ax = sns.countplot(
    data=movie_director_over_10,
    x="director",
    order=movie_director_over_10["director"].value_counts().index,
    palette="Set2"
)

plt.title("Distribución por Directores con entre 10 y 8 películas")
ax.tick_params(axis='x', rotation=90, labelsize=5)
plt.tight_layout()
plt.show()

### Autores

In [ ]:
#Autores
plt.figure(figsize=(12, 6))
ax = sns.countplot(
    data=movie_author,
    x="author",
    order=movie_author["author"].value_counts().index,
    palette="Set2"
)

plt.title("Distribución por Autores")
ax.tick_params(axis='x', rotation=90, labelsize=8)
plt.tight_layout()
plt.show()

Al tener tantos autores, nos enfocaremos en aquellos con más de 10 películas.

In [ ]:
author_counts = movie_author['author'].value_counts()

top_authors = author_counts[author_counts > 10].index

# Filtrar el dataset
movie_author_top = movie_author[movie_author['author'].isin(top_authors)]

# Gráfico
plt.figure(figsize=(12, 6))
ax = sns.countplot(
    data=movie_author_top,
    x="author",
    order=movie_author_top["author"].value_counts().index,
    palette="Set2"
)

plt.title("Distribución por Autores con >10 películas")
ax.tick_params(axis='x', rotation=90, labelsize=5)
plt.tight_layout()
plt.show()

Para el dataset de críticas, usaremos otro enfoque.

#### 3.2.1. Dividir el Dataset en categoricos y numericos

In [ ]:
c_df_copy = critics.copy()
c_df_copy.dtypes.unique()

Todas las variables son categoricas.

#### 3.2.2 Análisis sober variables categóricas

In [ ]:
print(c_df_copy['rotten_tomatoes_link'].value_counts().head(10))
print(f"Total de productos únicos: {c_df_copy['rotten_tomatoes_link'].nunique()}")
print("------------------------------------------------------")
print(c_df_copy['publisher_name'].value_counts().head(10))
print(f"Total de editoriales únicos: {c_df_copy['publisher_name'].nunique()}")

In [ ]:
categorical_cols = [ 'top_critic', 'review_type']


for col in categorical_cols:
    c_df_copy[col].value_counts().plot(kind='bar', figsize=(6,4), title=col)
    plt.show()

- La mayoría de críticos no son destacados en una proporción casi dos veces más que los que sí lo son.
- Las reseñas positivas son casi el doble de las negativas.

In [ ]:
top_n_peliculas = 10
top_n_editorial = 10

fig, axis = plt.subplots(1, 2, figsize=(16, 6))

# PRODUCTOS
top_products = c_df_copy['rotten_tomatoes_link'].value_counts().head(top_n_peliculas).index
sns.countplot(ax=axis[0], data=c_df_copy, x='rotten_tomatoes_link', order=top_products)
axis[0].set_title("Top 10 Peliculas reseñadas")
axis[0].tick_params(axis='x', rotation=90)
print("Top 10 peliculas graficados:", len(top_products))

# USUARIOS
top_users = c_df_copy['publisher_name'].value_counts().head(top_n_editorial).index
sns.countplot(ax=axis[1], data=c_df_copy, x='publisher_name', order=top_users)
axis[1].set_title("Top 10 Editoriales")
axis[1].tick_params(axis='x', rotation=45)
print("Top 10 editoriales graficados:", len(top_users))

plt.tight_layout()
plt.show()

- El top de peliculas reseñadas esta rodeando las 500 reseñas.
- La editorial con mas reseñas de películas es New York Times con casi 12000, avanzando de manera descendente hasta Chicado Sun-Times con casi 8000.

## 4. Análisis Multivariante

### 4.1 Análisis Numérico - Numérico

In [ ]:
fig, axis = plt.subplots(5, 2, figsize = (15, 20))

# Crear un diagrama de dispersión múltiple

sns.regplot(ax = axis[0, 0], data = movies, x = "audience_rating", y = "tomatometer_rating").set(ylabel=None)
sns.heatmap(movies[["tomatometer_rating", "audience_rating"]].corr(), annot = True, fmt = ".2f", ax = axis[0, 1])

sns.regplot(ax = axis[1, 0], data = movies, x = "tomatometer_fresh_critics_count", y = "tomatometer_rating").set(ylabel=None)
sns.heatmap(movies[["tomatometer_rating", "tomatometer_fresh_critics_count"]].corr(), annot = True, fmt = ".2f", ax = axis[1, 1])

sns.regplot(ax = axis[2, 0], data = movies, x = "tomatometer_rotten_critics_count", y = "tomatometer_rating").set(ylabel=None)
sns.heatmap(movies[["tomatometer_rating", "tomatometer_rotten_critics_count"]].corr(), annot = True, fmt = ".2f", ax = axis[2, 1])

sns.regplot(ax = axis[3, 0], data = movies, x = "tomatometer_fresh_critics_count", y = "audience_rating").set(ylabel=None)
sns.heatmap(movies[["audience_rating", "tomatometer_fresh_critics_count"]].corr(), annot = True, fmt = ".2f", ax = axis[3, 1])

sns.regplot(ax = axis[4, 0], data = movies, x = "tomatometer_rotten_critics_count", y = "audience_rating").set(ylabel=None)
sns.heatmap(movies[["audience_rating", "tomatometer_rotten_critics_count"]].corr(), annot = True, fmt = ".2f", ax = axis[4, 1])

# Ajustar el layout
plt.tight_layout()

# Mostrar el plot
plt.show()

### 4.2 Análisis Categórico - Categórico

In [ ]:
def cat_cat(df, cat1, cat2, top_n_cat1=None, top_n_cat2=None):
    
    if top_n_cat1:
        top_values1 = df[cat1].value_counts().nlargest(top_n_cat1).index
        df = df[df[cat1].isin(top_values1)]
    if top_n_cat2:
        top_values2 = df[cat2].value_counts().nlargest(top_n_cat2).index
        df = df[df[cat2].isin(top_values2)]
    
    ct = pd.crosstab(df[cat1], df[cat2])
    
    ct_prop = ct.div(ct.sum(axis=1).replace(0,1), axis=0)
    
    ct_prop.plot(kind='bar', stacked=True, figsize=(12,6))
    plt.xlabel(cat1)
    plt.ylabel('Proporción')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title=cat2, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
movies_exploded_dir = movies.explode('directors')

movies_exploded_dir = movies_exploded_dir[movies_exploded_dir['directors'] != '']
movies_exploded_dir = movies_exploded_dir.dropna(subset=['directors'])

top10_directors = (
    movies_exploded_dir['directors']
    .value_counts()
    .head(10)
    .index
)
movies_top_directors = movies_exploded_dir[movies_exploded_dir['directors'].isin(top10_directors)]

In [ ]:
movies_exploded_aut = movies.explode('authors')

movies_exploded_aut = movies_exploded_aut[movies_exploded_aut['authors'] != '']
movies_exploded_aut = movies_exploded_aut.dropna(subset=['authors'])

top10_authors = (
    movies_exploded_aut['authors']
    .value_counts()
    .head(10)
    .index
)
movies_top_authors = movies_exploded_aut[movies_exploded_aut['authors'].isin(top10_authors)]

In [ ]:
movies_exploded_act = movies.explode('actors')

movies_exploded_act = movies_exploded_act[movies_exploded_act['actors'] != '']
movies_exploded_act = movies_exploded_act.dropna(subset=['actors'])

top10_actors = (
    movies_exploded_act['actors']
    .value_counts()
    .head(10)
    .index
)
movies_top_actors = movies_exploded_act[movies_exploded_act['actors'].isin(top10_actors)]

In [ ]:
movies_exploded_genres = movies.explode('genres')

cat_cat(
    df=movies_exploded_genres,
    cat1='genres',        
    cat2='content_rating',
    top_n_cat1=None,      
    top_n_cat2=None       
)


In [ ]:
cat_cat(
    df=movies_exploded_genres,
    cat1='genres',        
    cat2='tomatometer_status',
    top_n_cat1=None,      
    top_n_cat2=None       
)

In [ ]:
cat_cat(
    df=movies_exploded_genres,
    cat1='genres',        
    cat2='audience_status',
    top_n_cat1=None,      
    top_n_cat2=None       
)

In [ ]:
cat_cat(
    df=movies_top_directors,
    cat1='content_rating',        
    cat2='directors',
    top_n_cat1=None,      
    top_n_cat2=None       
)


In [ ]:
cat_cat(
    df=movies_top_authors,
    cat1='content_rating',        
    cat2='authors',
    top_n_cat1=None,      
    top_n_cat2=None       
)

In [ ]:
cat_cat(
    df=movies_top_actors,
    cat1='content_rating',        
    cat2='actors',
    top_n_cat1=None,      
    top_n_cat2=None       
)

In [ ]:
cat_cat(c_df_copy, 'rotten_tomatoes_link', 'publisher_name', top_n_cat1=top_n_peliculas, top_n_cat2=top_n_editorial)

Parece que eFilmCritic.com es quien más reseñas hace a las peliculas con más reseñas del top.

In [ ]:
cat_cat(c_df_copy, 'top_critic', 'publisher_name', top_n_cat2=20)

Se ve que la mayoría de las editoriales con mas reseñas tienen críticos cuya opinión es validada, es importante teniendo en cuenta que hay más con la categoría false que con true.

In [ ]:
cat_cat(c_df_copy, 'review_type', 'rotten_tomatoes_link', top_n_cat2=20)

Las buenas reseñas estan distribuidas mas equitativamente entre las películas mientras que las malas se acumulan, esto sugiere que se le da importancia a que hay películas con más reseñas malas que buenas.

### 4.3 Análisis Numérico - Categórico

### 4.3.1 Matriz de Correlación Múltiple

Para continuar con el análisis de las variables categóricas y poder hacer un gráfico de correlaciones, debemos transformar estas variables en variables numéricas.

In [ ]:
movies_n = movies.copy()

In [ ]:
movies_n["content_rating_n"] = pd.factorize(movies_n["content_rating"])[0]
movies_n["production_company_n"] = pd.factorize(movies_n["production_company"])[0]
movies_n["tomatometer_status_n"] = pd.factorize(movies_n["tomatometer_status"])[0]
movies_n["audience_status_n"] = pd.factorize(movies_n["audience_status"])[0]

In [ ]:
cols = [
    "content_rating_n",
    "release_year",
    "streaming_release_year",
    "runtime",
    "production_company_n",
    "tomatometer_status_n",
    "tomatometer_rating",
    "tomatometer_count",
    "audience_status_n",
    "audience_rating",
    "audience_count",
    "tomatometer_top_critics_count",
    "tomatometer_fresh_critics_count",
    "tomatometer_rotten_critics_count"
]

movies_corr = movies_n[cols]

movies_corr = movies_corr.fillna(0)


In [ ]:
corr_matrix = movies_corr.corr()

fig, axis = plt.subplots(figsize = (15, 15))

sns.heatmap(corr_matrix, annot = True, fmt = ".2f")

plt.tight_layout()

plt.show()

## 5. Ingeniería de características

### 5.1. Preparar los datos

Como primera etapa vamos a mapear las reseñas de frescas o podridas como boolean

In [ ]:
c_df_copy['review_label'] = c_df_copy['review_type'].map({'Fresh': 1, 'Rotten': 0})

print(c_df_copy[['review_type', 'review_label']].head())
c_df_copy.drop(columns=['review_type'], inplace=True)

In [ ]:
c_df_copy.head()

In [ ]:
c_df_copy['top_critic_num'] = c_df_copy['top_critic'].astype(int)
c_df_copy.drop(columns=['top_critic'], inplace=True)

In [ ]:
c_df_copy['review_len'] = c_df_copy['review_content'].str.len()

# cantidad de palabras
c_df_copy['review_word_count'] = c_df_copy['review_content'].str.split().apply(len)

In [ ]:
c_df_copy.head()

### 5.2. Análisis de outliers

Sólo se tendrán en cuenta que las reseñas con una longitud mínima.

In [ ]:
c_df_copy = c_df_copy[(c_df_copy['review_len'] >= 10) & (c_df_copy['review_len'] <= 5000)]

In [ ]:
num_cols = ['review_len', 'review_word_count', 'top_critic_num', 'review_label']

for col in num_cols:
    plt.figure(figsize=(8,2))
    sns.boxplot(x=c_df_copy[col])
    plt.title(f'Boxplot de {col}')
    plt.show()

### 5.3. Dividir el train/test de marcos de datos

In [ ]:
X = c_df_copy['review_content']  
y = c_df_copy['review_label']    

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 6. Transformar el texto en matriz

In [ ]:
vec_model = CountVectorizer(stop_words = "english", max_features=5000)
X_train_vec = vec_model.fit_transform(X_train)
X_test_vec = vec_model.transform(X_test)

## 7. Construir el Modelo usando Naive Bayes

In [ ]:
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)

y_pred_nb = nb_model.predict(X_test_vec)
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))

## 8. Optimización

In [ ]:
def evaluar_modelo(model, X_train_vec, X_test_vec, y_train, y_test, nombre="Modelo"):
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    acc = accuracy_score(y_test, y_pred)
    print(f"{nombre} Accuracy: {acc:.5f}")
    return model, y_pred

In [ ]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

hyperparams = {
    'vectorizer__max_features': [1000, 2000, 3000, None],
    'vectorizer__ngram_range': [(1,1), (1,2)],
    'nb__alpha': np.linspace(0.1, 2.0, 20)
}

In [ ]:
# Naive Bayes simple
vec_model = CountVectorizer(stop_words="english")
X_train_vec = vec_model.fit_transform(X_train)
X_test_vec  = vec_model.transform(X_test)

nb_model = MultinomialNB()
nb_model, y_pred_nb = evaluar_modelo(nb_model, X_train_vec, X_test_vec, y_train, y_test, "Naive Bayes")

In [ ]:
# 5. Naive Bayes optimizado con Pipeline + RandomizedSearchCV
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

hyperparams = {
    'vectorizer__max_features': [1000, 2000, 3000, None],
    'vectorizer__ngram_range': [(1,1), (1,2)],
    'nb__alpha': np.linspace(0.1, 2.0, 20)
}

grid = RandomizedSearchCV(pipeline, hyperparams, scoring="accuracy", n_iter=20, random_state=42)
grid.fit(X_train, y_train)

best_nb_model = grid.best_estimator_
y_pred_best_nb = best_nb_model.predict(X_test)

print("Naive Bayes Optimizado Accuracy:", accuracy_score(y_test, y_pred_best_nb))
print("Mejores parámetros:", grid.best_params_)

In [ ]:
#Logistic Regression
log_model = LogisticRegression(max_iter=1000)
log_model, y_pred_log = evaluar_modelo(log_model, X_train_vec, X_test_vec, y_train, y_test, "Logistic Regression")

In [ ]:
acc_nb = accuracy_score(y_test, y_pred_best_nb)
acc_log = accuracy_score(y_test, y_pred_log)

if acc_log > acc_nb:
    modelo_final = log_model
    nombre_final = "Logistic Regression"
else:
    modelo_final = best_nb_model
    nombre_final = "Naive Bayes Optimizado"

print(f"Mejor modelo: {nombre_final} ({max(acc_nb, acc_log):.5f})")

In [ ]:
pipeline = Pipeline([
    ("vectorizer", CountVectorizer(stop_words="english")),
    ("classifier", modelo_final)   
])

pipeline.fit(X_train, y_train)

# Guardar el pipeline completo
with open("../models/rotten_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)

print("Pipeline con modelo final guardado correctamente.")

## 9. Combinar Datasets

Para el sistema de recomendación, necesitaremos trabajar con un único modelo, por lo cual agregaremos al dataset *movies* una columna adicional que contenga el valor promedio del puntaje obtenido al hacer el análisis de sentimientos.

In [ ]:
critics["review_content"] = critics["review_content"].fillna("")

critics["sentiment_prob"] = pipeline.predict_proba(critics["review_content"])[:, 1]

In [ ]:
# Aqui se argupan por el id y se saca la media
sentiment_scores = critics.groupby("rotten_tomatoes_link")["sentiment_prob"].mean().reset_index()
sentiment_scores.rename(columns={"sentiment_prob": "avg_sentiment_score"}, inplace=True)

# Unir al dataset de películas
movies = movies.merge(sentiment_scores, on="rotten_tomatoes_link", how="left")

In [ ]:
movies.head()

## 10. Guardar Dataset

In [ ]:
movies.to_csv("../data/processed/movies.csv", index=False)